In [1]:
import base64
import requests

# OpenAI API Key
api_key = "xxxxx"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [2]:
import pandas as pd

base_path = "clear/apartment_images/"
gt_path = 'ground_truth.csv'

# convert list of images to path
def image_list(folder_id, image_string):
    images = []
    array = image_string.split(';')
    for item in array:
        image_url = base_path + str(folder_id) + '/image_' + item + '.jpg'
        images.append(image_url)
    return images


In [6]:
# get building age for each row in order to get training ids
def get_building_gt_age():
    df = pd.read_csv(gt_path)
    for index, row in df.iterrows():
        raw_image_list = row['age image']
        actual_age = row['building age']
        if pd.isna(actual_age) or actual_age == '':
            actual_age = row['raw age data']
        print(index, ',', actual_age)

get_building_gt_age()

0 , 2014
1 , 2013
2 , 2015
3 , 2014
4 , 2007-2011
5 , 2015
6 , 2014
7 , 2016
8 , 2008
9 , 2017
10 , 2015
11 , 19th century
12 , 1930-1949
13 , 1996-2002
14 , 2015
15 , 1900-1929
16 , 2021
17 , 2003-2006
18 , 2020
19 , 1991-1995
20 , before 1900
21 , 2018
22 , 1997-2000
23 , 2016
24 , 2016
25 , 2007-2011
26 , 2003-2006
27 , 1900-1929
28 , 2011
29 , 2019
30 , 1996-2002
31 , 1996-2002
32 , 2020
33 , 2021
34 , 2003-2006
35 , 2016
36 , 2016
37 , 2019
38 , 2015
39 , 2017
40 , 2015
41 , 2005
42 , 2017
43 , 2015
44 , 2008
45 , 2009
46 , 2023


In [7]:
# get training ids
import random

# single instance ids (these only appear once in the dataset, so they need to be in the training data set)
single_instance_training_sample = [1, 11, 12, 19, 20, 21, 22, 28, 41, 45, 46]
print(single_instance_training_sample)

[1, 11, 12, 19, 20, 21, 22, 28, 41, 45, 46]


In [10]:
yr_2008_ids = [8, 44]
yr_2008_training_sample = random.sample(yr_2008_ids, 1) 
print(yr_2008_training_sample)

[44]


In [11]:
yr_2014_ids = [0, 3, 6]
yr_2014_training_sample = random.sample(yr_2014_ids, 1) 
print(yr_2014_training_sample)

[0]


In [24]:
yr_2015_ids = [2, 5, 10, 14, 38, 40, 43]
yr_2015_training_sample = random.sample(yr_2015_ids, 4) 
print(yr_2015_training_sample)

[10, 14, 43, 2]


In [25]:
yr_2016_ids = [7, 23, 24, 35, 36]
yr_2016_training_sample = random.sample(yr_2016_ids, 3) 
print(yr_2016_training_sample)

[36, 24, 35]


In [16]:
yr_2017_ids = [9, 39, 42]
yr_2017_training_sample = random.sample(yr_2017_ids, 1) 
print(yr_2017_training_sample)

[39]


In [17]:
yr_2019_ids = [29, 37]
yr_2019_training_sample = random.sample(yr_2019_ids, 1) 
print(yr_2019_training_sample)

[37]


In [18]:
yr_2020_ids = [18, 32]
yr_2020_training_sample = random.sample(yr_2020_ids, 1) 
print(yr_2020_training_sample)

[18]


In [19]:
yr_2021_ids = [16, 33]
yr_2021_training_sample = random.sample(yr_2021_ids, 1) 
print(yr_2021_training_sample)

[16]


In [20]:
yr_1900_1929_ids = [15, 27]
yr_1900_1929_training_sample = random.sample(yr_1900_1929_ids, 1) 
print(yr_1900_1929_training_sample)

[27]


In [21]:
yr_1996_2002_ids = [13, 30, 31]
yr_1996_2002_training_sample = random.sample(yr_1996_2002_ids, 1) 
print(yr_1996_2002_training_sample)

[30]


In [22]:
yr_2003_2006_ids = [17, 26, 34]
yr_2003_2006_training_sample = random.sample(yr_2003_2006_ids, 1) 
print(yr_2003_2006_training_sample)

[34]


In [23]:
yr_2007_2011_ids = [4, 25]
yr_2007_2011_training_sample = random.sample(yr_2007_2011_ids, 1) 
print(yr_2007_2011_training_sample)

[4]


In [37]:
# Combine all lists and check for duplicates, there shouldn't be!
age_samples_combined = single_instance_training_sample + yr_2008_training_sample + yr_2014_training_sample + yr_2015_training_sample + yr_2016_training_sample + yr_2017_training_sample + yr_2019_training_sample + yr_2020_training_sample + yr_2021_training_sample + yr_1900_1929_training_sample + yr_1996_2002_training_sample + yr_2003_2006_training_sample + yr_2007_2011_training_sample
duplicates = [item for item in set(age_samples_combined) if age_samples_combined.count(item) > 1]
print("Duplicates:", duplicates)

Duplicates: []


In [46]:
# Sort them by increasing id
age_training_sample = sorted(age_samples_combined)
print(age_training_sample)
print(len(age_training_sample))

[0, 1, 2, 4, 10, 11, 12, 14, 16, 18, 19, 20, 21, 22, 24, 27, 28, 30, 34, 35, 36, 37, 39, 41, 43, 44, 45, 46]
28


In [28]:
from openai import OpenAI
client = OpenAI(api_key=api_key)
def get_completion(prompt):
    response = client.chat.completions.create(
      model="gpt-4o-2024-08-06", #"gpt-4o",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt,
            },
          ],
        }
      ],
      max_tokens=4000,
    )
    # print(response)
    return response.choices[0].message.content

In [48]:
# whole set id
id_year_built = [
  {"id": 0, "year_built": "2014"},
  {"id": 1, "year_built": "2013"},
  {"id": 2, "year_built": "2015"},
  {"id": 3, "year_built": "2014"},
  {"id": 4, "year_built": "2007-2011"},
  {"id": 5, "year_built": "2015"},
  {"id": 6, "year_built": "2014"},
  {"id": 7, "year_built": "2016"},
  {"id": 8, "year_built": "2008"},
  {"id": 9, "year_built": "2017"},
  {"id": 10, "year_built": "2015"},
  {"id": 11, "year_built": "19th century"},
  {"id": 12, "year_built": "1930-1949"},
  {"id": 13, "year_built": "1996-2002"},
  {"id": 14, "year_built": "2015"},
  {"id": 15, "year_built": "1900-1929"},
  {"id": 16, "year_built": "2021"},
  {"id": 17, "year_built": "2003-2006"},
  {"id": 18, "year_built": "2020"},
  {"id": 19, "year_built": "1991-1995"},
  {"id": 20, "year_built": "before 1900"},
  {"id": 21, "year_built": "2018"},
  {"id": 22, "year_built": "1997-2000"},
  {"id": 23, "year_built": "2016"},
  {"id": 24, "year_built": "2016"},
  {"id": 25, "year_built": "2007-2011"},
  {"id": 26, "year_built": "2003-2006"},
  {"id": 27, "year_built": "1900-1929"},
  {"id": 28, "year_built": "2011"},
  {"id": 29, "year_built": "2019"},
  {"id": 30, "year_built": "1996-2002"},
  {"id": 31, "year_built": "1996-2002"},
  {"id": 32, "year_built": "2020"},
  {"id": 33, "year_built": "2021"},
  {"id": 34, "year_built": "2003-2006"},
  {"id": 35, "year_built": "2016"},
  {"id": 36, "year_built": "2016"},
  {"id": 37, "year_built": "2019"},
  {"id": 38, "year_built": "2015"},
  {"id": 39, "year_built": "2017"},
  {"id": 40, "year_built": "2015"},
  {"id": 41, "year_built": "2005"},
  {"id": 42, "year_built": "2017"},
  {"id": 43, "year_built": "2015"},
  {"id": 44, "year_built": "2008"},
  {"id": 45, "year_built": "2009"},
  {"id": 46, "year_built": "2023"}
]

# get training set id rows
training_set_rows = []
for item in id_year_built:
    if item['id'] in age_training_sample:
        training_set_rows.append({'id': item['id'], 'year_built': item['year_built']})
print(training_set_rows)
                                 
prompt =  f"""
You are a surveyor. You are given this list of buildings, each row is a building with their id and the year they are built {training_set_rows}. First group the buildings by 3 eras to ensure good coverage representative of the architectural style and dataset, then return the ids of buildings per era in an array.
"""

result = get_completion(prompt)
print(result)

[{'id': 0, 'year_built': '2014'}, {'id': 1, 'year_built': '2013'}, {'id': 2, 'year_built': '2015'}, {'id': 4, 'year_built': '2007-2011'}, {'id': 10, 'year_built': '2015'}, {'id': 11, 'year_built': '19th century'}, {'id': 12, 'year_built': '1930-1949'}, {'id': 14, 'year_built': '2015'}, {'id': 16, 'year_built': '2021'}, {'id': 18, 'year_built': '2020'}, {'id': 19, 'year_built': '1991-1995'}, {'id': 20, 'year_built': 'before 1900'}, {'id': 21, 'year_built': '2018'}, {'id': 22, 'year_built': '1997-2000'}, {'id': 24, 'year_built': '2016'}, {'id': 27, 'year_built': '1900-1929'}, {'id': 28, 'year_built': '2011'}, {'id': 30, 'year_built': '1996-2002'}, {'id': 34, 'year_built': '2003-2006'}, {'id': 35, 'year_built': '2016'}, {'id': 36, 'year_built': '2016'}, {'id': 37, 'year_built': '2019'}, {'id': 39, 'year_built': '2017'}, {'id': 41, 'year_built': '2005'}, {'id': 43, 'year_built': '2015'}, {'id': 44, 'year_built': '2008'}, {'id': 45, 'year_built': '2009'}, {'id': 46, 'year_built': '2023'}]
T

In [53]:
# pick 3 random ids from different eras to use for feature extraction

era1 = random.choice([20, 11, 27])
print('era1', era1)

era1 20


In [50]:
era2 = random.choice([12, 19, 22, 30])
print('era2', era2)

era2 22


In [51]:
era3 = random.choice([0, 1, 2, 4, 10, 14, 16, 18, 21, 24, 28, 34, 35, 36, 37, 39, 41, 43, 44, 45, 46])
print('era3', era3)

era3 28


In [63]:
custom_prompt = "Your task is to provide a detailed label of every architectural feature for the building that will help determine the age of the building whether it is before 1900, 1900-1930, 1930-1950, 1950-1970, 1970-1990, 1990-2020, 2020-now. List 50 visible features that are significant for building age."
image_type = 'age image' # change this for age, window etc. this is used to determine which images to use
def llm_get_features(images):

    prompt = f"""

    You are a surveyor. You are given a set of images that belong to the same building.
    
    {custom_prompt}
    
    The building is located in UK. Return the features as a list.
    
    """

    content = [
            {
              "type": "text",
              "text": prompt
            }
          ]

    for image in images:
        base64_image = encode_image(image)
        content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": content
        }
      ],
      "max_tokens": 5000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # print(response.text)
    print(response.json()['choices'][0]['message']['content'])


def extract_features(row_id):
    df = pd.read_csv(gt_path)
    # for index, row in df.iterrows():
        # if index in [0, 11, 22, 46]:
    raw_image_list = df.loc[row_id][image_type]
    print('\n\n\n---row', row_id, df.loc[row_id]['Address'], df.loc[row_id]['Thesquare Building URL'])
    llm_get_features(image_list(row_id, raw_image_list))
    # actual_age = row['building age']
    # if pd.isna(actual_age) or actual_age == '':
    #     actual_age = row['raw age data']
    # print('---answer---\n', actual_age)

In [64]:
extract_features(20)




---row 20 Cleveland Residences , 12-14 Cleveland Street , W1T 4HY https://www.thesqua.re/london/serviced-apartments/cleveland-residences-fitzrovia-serviced-apartments-59087712
To assess the age of the building from the architectural features visible in these images, here’s a list of 50 characteristics to consider:

1. Brickwork style
2. Window design and shape
3. Sash windows
4. Arched windows
5. Decorative lintels
6. Roof type and materials
7. Chimney presence and style
8. Gable ends
9. Facade symmetry
10. Iron railings
11. Door style and material
12. Door frames and transoms
13. External cornice details
14. Stone or brick quoins
15. Street-facing facade
16. Proportions of windows to walls
17. Building ornamentation
18. String courses (horizontal bands)
19. Terraced layout
20. Parapet walls
21. Mouldings around windows
22. Basement windows
23. Skylights or dormers
24. Roofing tiles or slates
25. Use of stucco or render
26. Brick color and texture
27. Rainwater goods (gutters and do

In [65]:
extract_features(22)




---row 22 182-205 Imperial Dr , Rayners Lane , HA2 7DP https://www.thesqua.re/london/serviced-apartments/harrow-apartments-by-mysquare-1240367693
Here is a list of architectural features visible in the images that could help determine the age of the building:

1. Large glass windows
2. Aluminum window frames
3. Modern blinds
4. Flat roof design
5. Minimalistic interior design
6. Whitewashed walls
7. Contemporary light fixtures
8. Laminate wood flooring
9. Sleek cabinetry
10. Stainless steel kitchen fittings
11. Open floor plan
12. UPVC doors
13. Thermally efficient radiators
14. Insulated external walls
15. Modular furniture
16. Built-in kitchen appliances
17. Recessed lighting
18. Energy-efficient windows
19. Double glazing
20. Simple geometric shapes
21. Absence of decorative moldings
22. Clean, straight lines
23. Monochrome color palette
24. Absence of ornamental detailing
25. Elevator access
26. Security camera installation points
27. Bicycle storage space
28. Fire sprinkler sys

In [66]:
extract_features(28)




---row 28 Cobalt Point, 38 Millharbour , 38 Millharbour , E14 9JU https://www.thesqua.re/london/serviced-apartments/lanterns-court-apartments-56217710
Here is a list of architectural features that can help determine the age of the building:

1. Window style and frame material
2. Floor-to-ceiling windows
3. Balcony design
4. Façade materials (e.g., concrete, glass)
5. Flat roof design
6. Use of metal cladding
7. Presence of a curtain wall system
8. Shared amenity spaces
9. Open-plan interior layout
10. Minimalist interior design
11. Integrated appliances
12. Energy-efficient lighting
13. Modern ventilation systems
14. Neutral color palette
15. Use of sustainable materials
16. Double-glazing
17. Contemporary door styles
18. Lift access
19. Reinforced concrete structure
20. Fire safety signage
21. Integrated storage solutions
22. Underfloor heating
23. Insulation types
24. Use of synthetic building materials
25. Smart home technology
26. Sliding glass doors
27. Recessed lighting
28. Ph

In [67]:
age_features_1 = [
    "Brickwork style",
    "Window design and shape",
    "Sash windows",
    "Arched windows",
    "Decorative lintels",
    "Roof type and materials",
    "Chimney presence and style",
    "Gable ends",
    "Facade symmetry",
    "Iron railings",
    "Door style and material",
    "Door frames and transoms",
    "External cornice details",
    "Stone or brick quoins",
    "Street-facing facade",
    "Proportions of windows to walls",
    "Building ornamentation",
    "String courses (horizontal bands)",
    "Terraced layout",
    "Parapet walls",
    "Mouldings around windows",
    "Basement windows",
    "Skylights or dormers",
    "Roofing tiles or slates",
    "Use of stucco or render",
    "Brick color and texture",
    "Rainwater goods (gutters and downpipes)",
    "Period-style lighting fixtures",
    "External color scheme",
    "Balcony design",
    "Door hardware (knockers, handles)",
    "Ventilation style (grills, fans)",
    "Interior fireplace design",
    "Skirting board design",
    "Ceiling height and details (e.g., coving)",
    "Architraves around doors",
    "Internal flooring materials",
    "Modern partition walls",
    "Light fittings and switches",
    "Built-in cabinetry style",
    "Heating system (grills, radiators)",
    "Decoration style and fixtures",
    "Kitchen and bathroom fittings",
    "Cornice or coving presence",
    "Plasterwork details",
    "Floorplan/layout",
    "Staircase design",
    "Internal door style",
    "Loft conversions or additions",
    "Overall building scale and massing"
]

In [68]:
age_features_2 = [
    "Large glass windows",
    "Aluminum window frames",
    "Modern blinds",
    "Flat roof design",
    "Minimalistic interior design",
    "Whitewashed walls",
    "Contemporary light fixtures",
    "Laminate wood flooring",
    "Sleek cabinetry",
    "Stainless steel kitchen fittings",
    "Open floor plan",
    "UPVC doors",
    "Thermally efficient radiators",
    "Insulated external walls",
    "Modular furniture",
    "Built-in kitchen appliances",
    "Recessed lighting",
    "Energy-efficient windows",
    "Double glazing",
    "Simple geometric shapes",
    "Absence of decorative moldings",
    "Clean, straight lines",
    "Monochrome color palette",
    "Absence of ornamental detailing",
    "Elevator access",
    "Security camera installation points",
    "Bicycle storage space",
    "Fire sprinkler systems",
    "Electronic security systems",
    "Central heating systems",
    "Modern ventilation units",
    "Motion sensor lighting",
    "Integrated waste disposal units",
    "High-efficiency water fixtures",
    "Eco-friendly construction materials",
    "Energy performance certificates",
    "Double-height lobby space",
    "Acoustic insulation",
    "Steel-reinforced concrete structures",
    "Solar panels (if present)",
    "Modern signage typography",
    "Use of synthetic materials",
    "Glass curtain walls",
    "External cladding",
    "Steel railing designs",
    "Balconies with glass balustrades",
    "Public access spaces",
    "Automated building management systems",
    "Contemporary landscaping",
    "Presence of digital infrastructures"
]

In [70]:
age_features_3 = [
    "Window style and frame material",
    "Floor-to-ceiling windows",
    "Balcony design",
    "Façade materials (e.g., concrete, glass)",
    "Flat roof design",
    "Use of metal cladding",
    "Presence of a curtain wall system",
    "Shared amenity spaces",
    "Open-plan interior layout",
    "Minimalist interior design",
    "Integrated appliances",
    "Energy-efficient lighting",
    "Modern ventilation systems",
    "Neutral color palette",
    "Use of sustainable materials",
    "Double-glazing",
    "Contemporary door styles",
    "Lift access",
    "Reinforced concrete structure",
    "Fire safety signage",
    "Integrated storage solutions",
    "Underfloor heating",
    "Insulation types",
    "Use of synthetic building materials",
    "Smart home technology",
    "Sliding glass doors",
    "Recessed lighting",
    "Photovoltaic cells",
    "Green building features",
    "Wheelchair accessibility",
    "Acoustic insulation",
    "Absence of decorative moldings",
    "Presence of security systems",
    "Automated window blinds",
    "Keyless entry systems",
    "Communal outdoor areas",
    "Parking facilities",
    "Modular construction techniques",
    "Ventilation vents",
    "Fire sprinkler system",
    "Glass balconies",
    "Urban integration",
    "Use of LED lighting",
    "Building management systems",
    "Presence of gyms or fitness centers",
    "Contemporary signage",
    "Minimal cornices",
    "Modern staircases",
    "Insulated cladding",
    "Digital connectivity features"
]

In [71]:
raw_feature_list = age_features_1 + age_features_2 + age_features_3

prompt =  f"""
I have a list of features: {raw_feature_list}. Your task is to remove duplicated items, including features semantically similar.
"""

result = get_completion(prompt)
print(result)

Here's a revised list of features with duplicates and semantically similar items removed:

1. Brickwork style
2. Window design and shape
3. Arched windows
4. Roof type and materials
5. Chimney presence and style
6. Gable ends
7. Facade symmetry
8. Iron railings
9. Door style and material
10. External cornice details
11. Stone or brick quoins
12. Proportions of windows to walls
13. Building ornamentation
14. String courses (horizontal bands)
15. Terraced layout
16. Parapet walls
17. Mouldings around windows
18. Basement windows
19. Skylights or dormers
20. Use of stucco or render
21. Brick color and texture
22. Rainwater goods (gutters and downpipes)
23. Period-style lighting fixtures
24. External color scheme
25. Balcony design
26. Door hardware (knockers, handles)
27. Interior fireplace design
28. Skirting board design
29. Ceiling height and details (e.g., coving)
30. Architraves around doors
31. Internal flooring materials
32. Modern partition walls
33. Light fittings and switches
34

In [77]:
unique_feature_list = f"""
1. Brickwork style
2. Window design and shape
3. Arched windows
4. Roof type and materials
5. Chimney presence and style
6. Gable ends
7. Facade symmetry
8. Iron railings
9. Door style and material
10. External cornice details
11. Stone or brick quoins
12. Proportions of windows to walls
13. Building ornamentation
14. String courses (horizontal bands)
15. Terraced layout
16. Parapet walls
17. Mouldings around windows
18. Basement windows
19. Skylights or dormers
20. Use of stucco or render
21. Brick color and texture
22. Rainwater goods (gutters and downpipes)
23. Period-style lighting fixtures
24. External color scheme
25. Balcony design
26. Door hardware (knockers, handles)
27. Interior fireplace design
28. Skirting board design
29. Ceiling height and details (e.g., coving)
30. Architraves around doors
31. Internal flooring materials
32. Modern partition walls
33. Light fittings and switches
34. Built-in cabinetry style
35. Heating system (grills, radiators)
36. Kitchen and bathroom fittings
37. Floorplan/layout
38. Staircase design
39. Internal door style
40. Loft conversions or additions
41. Overall building scale and massing
42. Large glass windows
43. Modern blinds
44. Minimalistic interior design
45. Contemporary light fixtures
46. Laminate wood flooring
47. Stainless steel kitchen fittings
48. Open floor plan
49. UPVC doors
50. Thermally efficient radiators
51. Insulated external walls
52. Modular furniture
53. Built-in kitchen appliances
54. Recessed lighting
55. Energy-efficient windows
56. Double glazing
57. Simple geometric shapes
58. Clean, straight lines
59. Monochrome color palette
60. Elevator access
61. Security systems
62. Bicycle storage space
63. Fire sprinkler systems
64. Central heating systems
65. Modern ventilation units
66. Motion sensor lighting
67. Integrated waste disposal units
68. High-efficiency water fixtures
69. Eco-friendly construction materials
70. Energy performance certificates
71. Double-height lobby space
72. Acoustic insulation
73. Steel-reinforced concrete structures
74. Solar panels
75. Modern signage typography
76. Use of synthetic materials
77. Glass curtain walls
78. External cladding
79. Steel railing designs
80. Balconies with glass balustrades
81. Public access spaces
82. Automated building management systems
83. Contemporary landscaping
84. Presence of digital infrastructures
85. Floor-to-ceiling windows
86. Façade materials (e.g., concrete, glass)
87. Shared amenity spaces
88. Integrated appliances
89. Energy-efficient lighting
90. Modern ventilation systems
91. Neutral color palette
92. Use of sustainable materials
93. Contemporary door styles
94. Lift access
95. Reinforced concrete structure
96. Fire safety signage
97. Integrated storage solutions
98. Underfloor heating
99. Insulation types
100. Smart home technology
101. Sliding glass doors
102. Photovoltaic cells
103. Green building features
104. Wheelchair accessibility
105. Keyless entry systems
106. Communal outdoor areas
107. Parking facilities
108. Modular construction techniques
109. Ventilation vents
110. Glass balconies
111. Urban integration
112. Use of LED lighting
113. Building management systems
114. Presence of gyms or fitness centers
115. Contemporary signage
116. Minimal cornices
117. Modern staircases
118. Insulated cladding
119. Digital connectivity features
"""
prompt =  f"""
I have a list of features: {unique_feature_list}. Your task is to cluster these features based on the type of feature. Aim to produce 8 clusters. 
"""

result = get_completion(prompt)
print(result)


Certainly! The features you've listed can be clustered into different categories based on their types and characteristics. Here's a suggested clustering into 8 groups:

1. **Architectural & Structural Features**:
   - Brickwork style
   - Roof type and materials
   - Chimney presence and style
   - Gable ends
   - Facade symmetry
   - Building ornamentation
   - String courses (horizontal bands)
   - Parapet walls
   - Use of stucco or render
   - Stone or brick quoins
   - External cornice details
   - Façade materials (e.g., concrete, glass)
   - External cladding
   - Steel-reinforced concrete structures
   - Reinforced concrete structure
   - Insulated external walls
   - Double-height lobby space

2. **Windows & Doors**:
   - Window design and shape
   - Arched windows
   - Door style and material
   - Proportions of windows to walls
   - Mouldings around windows
   - Skylights or dormers
   - Large glass windows
   - UPVC doors
   - Thermally efficient radiators
   - Energy-effic

In [78]:
categories = f"""
1. **Architectural & Structural Features**:
   - Brickwork style
   - Roof type and materials
   - Chimney presence and style
   - Gable ends
   - Facade symmetry
   - Building ornamentation
   - String courses (horizontal bands)
   - Parapet walls
   - Use of stucco or render
   - Stone or brick quoins
   - External cornice details
   - Façade materials (e.g., concrete, glass)
   - External cladding
   - Steel-reinforced concrete structures
   - Reinforced concrete structure
   - Insulated external walls
   - Double-height lobby space

2. **Windows & Doors**:
   - Window design and shape
   - Arched windows
   - Door style and material
   - Proportions of windows to walls
   - Mouldings around windows
   - Skylights or dormers
   - Large glass windows
   - UPVC doors
   - Thermally efficient radiators
   - Energy-efficient windows
   - Double glazing
   - Sliding glass doors
   - Floor-to-ceiling windows
   - Contemporary door styles

3. **Internal Features**:
   - Interior fireplace design
   - Skirting board design
   - Ceiling height and details (e.g., coving)
   - Architraves around doors
   - Staircase design
   - Internal door style
   - Loft conversions or additions
   - Built-in cabinetry style
   - Modern partition walls

4. **Modern & Minimalistic Design Elements**:
   - Minimalistic interior design
   - Clean, straight lines
   - Simple geometric shapes
   - Monochrome color palette
   - Neutral color palette

5. **Technological & Efficiency Features**:
   - Smart home technology
   - Thermally efficient radiators
   - Solar panels
   - Energy performance certificates
   - Automated building management systems
   - Presence of digital infrastructures
   - Digital connectivity features
   - Photovoltaic cells
   - Energy-efficient lighting

6. **Utilities & Technical Systems**:
   - Heating system (grills, radiators)
   - Modern ventilation units
   - Insulation types
   - Fire sprinkler systems
   - Central heating systems
   - Motion sensor lighting
   - Integrated waste disposal units
   - High-efficiency water fixtures
   - Underfloor heating
   - Ventilation vents
   - Modern ventilation systems

7. **Decorative & Ornamental Elements**:
   - Iron railings
   - Balcony design
   - Balconies with glass balustrades
   - Steel railing designs
   - Contemporary signage
   - Contemporary light fixtures
   - Modern blinds

8. **Community & Lifestyle Features**:
   - Communal outdoor areas
   - Public access spaces
   - Presence of gyms or fitness centers
   - Shared amenity spaces
   - Parking facilities
   - Wheelchair accessibility
   - Bicycle storage space
   - Keyless entry systems
"""

prompt =  f"""
Given this list {categories}, first clean the list to contain text only, then produce a python array, each subarray for each category.
"""

result = get_completion(prompt)
print(result)

To clean the list of text and organize it into a Python array of subarrays for each category, we first need to remove the category numbers and special characters, ensuring that we capture each feature under its relevant category. Here is the cleaned and organized list in Python array format:

```python
features = [
    [  # Architectural & Structural Features
        "Brickwork style",
        "Roof type and materials",
        "Chimney presence and style",
        "Gable ends",
        "Facade symmetry",
        "Building ornamentation",
        "String courses (horizontal bands)",
        "Parapet walls",
        "Use of stucco or render",
        "Stone or brick quoins",
        "External cornice details",
        "Façade materials (e.g., concrete, glass)",
        "External cladding",
        "Steel-reinforced concrete structures",
        "Reinforced concrete structure",
        "Insulated external walls",
        "Double-height lobby space",
    ],
    [  # Windows & Doors
      